The goal of this Notebook is to compile the Torchfort example outside the container, without having to recreate the container.

The original Dockerfile removes (`rm -rf torchfort`) the source files. You need to change it to keep the `/torchfort/build` directory, recreate the container in Docker, then copy `/torchfort/build` from inside the container to the external directory, in this case in the same location from where it was built.

In [1]:
! singularity --version

singularity-ce version 4.3.1-jammy


In [12]:
%%writefile sing
singularity exec --nv \
  --bind torchfort_local:/torchfort \
  ~/containers/torchfort.sif \
  bash -c "CUDA_PATH=/opt/nvidia/hpc_sdk/Linux_x86_64/25.3/cuda && \
    cd /torchfort/build && \
    $* "

Overwriting sing


In [13]:
%%writefile sing
singularity exec --nv \
  --bind ~/torchfort:/torchfort \
  ~/containers/torchfort.sif \
  bash -c "CUDA_PATH=/opt/nvidia/hpc_sdk/Linux_x86_64/25.3/cuda && \
    cd /torchfort/build && \
    $* "

Overwriting sing


In [14]:
! chmod +x sing

In [15]:
! ./sing "echo $PATH"

/home/x/conda/bin:/home/x/conda/condabin:/home/x/.nvm/versions/node/v24.3.0/bin:/home/x/.local/bin:/home/x/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/snap/bin:/home/x/bin:/home/x/.local/bin


In [16]:
! ./sing "cat /etc/os-release"

PRETTY_NAME="Ubuntu 22.04.5 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.5 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy


In [17]:
! ./sing "make -j10 train_distributed"

-- Found Python: /usr/bin/python3.10 (found version "3.10.12") found components: Interpreter 
-- Found CUDAToolkit: /opt/nvidia/hpc_sdk/Linux_x86_64/25.3/cuda/12.8/include (found version "12.8.93") 
-- NVHPC_CUDA_VERSION not specified.
-- Default CUDA version selected: 12.8
-- Using NCCL library: /usr/lib/x86_64-linux-gnu/libnccl.so
-- Found CUDAToolkit: /opt/nvidia/hpc_sdk/Linux_x86_64/25.3/cuda/12.8/include (found version "12.8.93") 
-- PyTorch: CUDA detected: 12.8
-- PyTorch: CUDA nvcc is: /opt/nvidia/hpc_sdk/Linux_x86_64/25.3/cuda/bin/nvcc
-- PyTorch: CUDA toolkit directory: /opt/nvidia/hpc_sdk/Linux_x86_64/25.3/cuda
-- PyTorch: Header version is: 12.8
CMake Warning at /usr/local/lib/python3.10/dist-packages/torch/share/cmake/Caffe2/public/cuda.cmake:140 (message):
  Failed to compute shorthash for libnvrtc.so
Call Stack (most recent call first):
  /usr/local/lib/python3.10/dist-packages/torch/share/cmake/Caffe2/Caffe2Config.cmake:86 (include)
  /usr/local/lib/python3.10/dist-packa

In [5]:
! ./sing "cd /torchfort && \
    nvfortran hello.f90 -o hello_world_exec"

In [24]:
! ls

0_notes-tfort#.txt	    torchfort_local
cmakelists.ipynb	    torchfort_local-2025-03-20
container-make.ipynb	    train-01.ipynb
dock-to-singu.ipynb	    train-02.ipynb
jl-insid-ctner.ipynb	    train_distrib-f90.ipynb
jl-insid-ctner-train.ipynb  train_distrib.ipynb
mpi-simple-example-fortran  train_distrib-sd.ipynb
openacc-tests.ipynb	    train_distrib-sd-slurm-02.ipynb
prompt-train_distr_02.txt   train_distrib-sd-slurm.ipynb
setup-nvptx.ipynb	    train_distrib.srm
sing			    train-sd.ipynb
tfort-singu-test-sd.ipynb   train-singu.ipynb


In [25]:
! ./sing "cd /torchfort && \
    ./hello_world_exec"

 Hello world!


## train_distr_02

In [27]:
%%writefile torchfort_local/examples/fortran/simulation/CMakeLists.txt
find_package(HDF5 COMPONENTS Fortran REQUIRED)

set(fortran_example_targets
  train
  train_distributed
#======================================
  train_distr_02
#======================================
)



add_executable(train)
target_sources(train
  PRIVATE
  train.f90
  simulation.f90
)
set_target_properties(train PROPERTIES Fortran_MODULE_DIRECTORY ${CMAKE_CURRENT_SOURCE_DIR}/mod/0 )

add_executable(train_distributed)
target_sources(train_distributed
  PRIVATE
  train_distributed.f90
  simulation.f90
)
set_target_properties(train_distributed PROPERTIES Fortran_MODULE_DIRECTORY ${CMAKE_CURRENT_SOURCE_DIR}/mod/1 )

#======================================
add_executable(train_distr_02)
target_sources(train_distr_02
  PRIVATE
  train_distr_02.f90
  simulation.f90
)
set_target_properties(train_distr_02 PROPERTIES Fortran_MODULE_DIRECTORY ${CMAKE_CURRENT_SOURCE_DIR}/mod/2 )
#======================================



foreach(tgt ${fortran_example_targets})
  target_include_directories(${tgt}
    PRIVATE
    ${CMAKE_BINARY_DIR}/include
    ${MPI_Fortran_INCLUDE_DIRS}
    ${HDF5_Fortran_INCLUDE_DIRS}
  )
  target_link_libraries(${tgt} PRIVATE MPI::MPI_Fortran)
  target_link_libraries(${tgt} PRIVATE hdf5::hdf5_fortran)
  target_link_libraries(${tgt} PRIVATE "${PROJECT_NAME}_fort")
  target_link_libraries(${tgt} PRIVATE ${PROJECT_NAME})
  if (CMAKE_Fortran_COMPILER_ID STREQUAL "NVHPC")
    target_compile_options(${tgt} PRIVATE $<$<COMPILE_LANGUAGE:Fortran>:-cpp -acc -gpu=${CUF_GPU_ARG}>)
    target_link_options(${tgt} PRIVATE $<$<COMPILE_LANGUAGE:Fortran>: -acc -gpu=${CUF_GPU_ARG}>)
  elseif (CMAKE_Fortran_COMPILER_ID STREQUAL "GNU")
    target_compile_options(${tgt} PRIVATE $<$<COMPILE_LANGUAGE:Fortran>:-cpp -fbackslash>)
  endif()
endforeach()

install(
  TARGETS ${fortran_example_targets}
  RUNTIME DESTINATION ${CMAKE_INSTALL_PREFIX}/bin/examples/fortran/simulation
)

install(
  FILES ${CMAKE_CURRENT_SOURCE_DIR}/config_mlp_native.yaml
        ${CMAKE_CURRENT_SOURCE_DIR}/config_fcn_torchscript.yaml
        ${CMAKE_CURRENT_SOURCE_DIR}/generate_fcn_model.py
        ${CMAKE_CURRENT_SOURCE_DIR}/visualize.py
  DESTINATION ${CMAKE_INSTALL_PREFIX}/bin/examples/fortran/simulation)

Overwriting torchfort_local/examples/fortran/simulation/CMakeLists.txt


In [26]:
! ./sing "cmake .. && \
        make -j10 train_distr_02"

-- Found Python: /usr/bin/python3.10 (found version "3.10.12") found components: Interpreter 
-- Found CUDAToolkit: /opt/nvidia/hpc_sdk/Linux_x86_64/25.3/cuda/12.8/include (found version "12.8.93") 
-- NVHPC_CUDA_VERSION not specified.
-- Default CUDA version selected: 12.8
-- Using NCCL library: /usr/lib/x86_64-linux-gnu/libnccl.so
-- Found CUDAToolkit: /opt/nvidia/hpc_sdk/Linux_x86_64/25.3/cuda/12.8/include (found version "12.8.93") 
-- PyTorch: CUDA detected: 12.8
-- PyTorch: CUDA nvcc is: /opt/nvidia/hpc_sdk/Linux_x86_64/25.3/cuda/bin/nvcc
-- PyTorch: CUDA toolkit directory: /opt/nvidia/hpc_sdk/Linux_x86_64/25.3/cuda
-- PyTorch: Header version is: 12.8
CMake Warning at /usr/local/lib/python3.10/dist-packages/torch/share/cmake/Caffe2/public/cuda.cmake:140 (message):
  Failed to compute shorthash for libnvrtc.so
Call Stack (most recent call first):
  /usr/local/lib/python3.10/dist-packages/torch/share/cmake/Caffe2/Caffe2Config.cmake:86 (include)
  /usr/local/lib/python3.10/dist-packa

In [7]:
! ./sing "make -j10 train_distr_02"

[  5%] Built target torchfort_fort
[ 91%] Built target torchfort
[ 94%] Linking Fortran executable train_distr_02
[100%] Built target train_distr_02
